<a href="https://colab.research.google.com/github/nilaynishant/AIMLTutorial/blob/main/TorchGeobasics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchgeo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.5/378.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.7/705.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.7/670.7 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 90.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
import os
import tempfile
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchgeo.datasets import NAIP,RasterDataset, ChesapeakeDE, stack_samples
from torchgeo.samplers import RandomGeoSampler, BatchGeoSampler, GridGeoSampler
from torchgeo.datasets.utils import download_url


In [3]:
naip_root = os.path.join(tempfile.gettempdir(), "naip")
naip_url = (
    "https://naipeuwest.blob.core.windows.net/naip/v002/de/2018/de_060cm_2018/38075/"
)
tiles = [
    "m_3807511_ne_18_060_20181104.tif",
    "m_3807511_se_18_060_20181104.tif",
    "m_3807512_nw_18_060_20180815.tif",
    "m_3807512_sw_18_060_20180815.tif",
]
for tile in tiles:
    download_url(naip_url + tile, naip_root)

naip = NAIP(naip_root)

100%|██████████| 513332284/513332284 [00:53<00:00, 9603584.59it/s]


100%|██████████| 521985441/521985441 [00:45<00:00, 11396331.69it/s]


100%|██████████| 489865657/489865657 [00:45<00:00, 10807855.90it/s]


100%|██████████| 484476647/484476647 [00:53<00:00, 9070351.13it/s]


In [4]:
chesapeake_root = os.path.join(tempfile.gettempdir(), "chesapeake")
os.makedirs(chesapeake_root, exist_ok=True)
chesapeake = ChesapeakeDE(chesapeake_root, crs=naip.crs, res=naip.res, download=True)

100%|██████████| 287350495/287350495 [00:11<00:00, 25720054.09it/s]


In [5]:
dataset = naip&chesapeake


In [6]:
sampler = RandomGeoSampler(dataset, size=256, length=500)

image_tensors = DataLoader(dataset, sampler=sampler, batch_size=10, collate_fn=stack_samples)


In [ ]:
i = 0
for batch in image_tensors:
  if i>0:
    break
  print( batch['mask'].shape)
  i=i+1

torch.Size([10, 1, 256, 256])


In [ ]:
random_images = iter(image_tensors)
data = next(random_images)
images = data['label']
print(images)

[]


In [ ]:
random_images = iter(image_tensors)
data = next(random_images)
images = data['image']
print(data['image'].shape)
conv1 = nn.Conv2d(4, 32 , (3,3))
x = conv1(images)
print(x.shape)
maxpool = nn.MaxPool2d(2, 2)
x = maxpool(x)
print(x.shape)
conv2 = nn.Conv2d(32, 32, (3,3))
x = conv2(x)
print(x.shape)
x=maxpool(x)
print(x.shape)
flatten=nn.Flatten()
x = flatten(x)
print(x.shape)
linear1 = nn.Linear(32*62*62, 120)
x = linear1(x)
print(x.shape)

torch.Size([10, 4, 256, 256])
torch.Size([10, 32, 254, 254])
torch.Size([10, 32, 127, 127])
torch.Size([10, 32, 125, 125])
torch.Size([10, 32, 62, 62])
torch.Size([10, 123008])
torch.Size([10, 120])


In [ ]:
random_images = iter(dataloader)
data = next(random_images)
images = data['image']
print(data['image'].shape)
conv1 = nn.Conv2d(4, 32 , (3,3))
x = conv1(images)
print(x.shape)
maxpool = nn.MaxPool2d(2, 2)
x = maxpool(x)
print(x.shape)
conv2 = nn.Conv2d(32, 32, (3,3))
x = conv2(x)
print(x.shape)
x=maxpool(x)
print(x.shape)
flatten=nn.Flatten()
x = flatten(x)
print(x.shape)
linear1 = nn.Linear(32*62*62, 120)
x = linear1(x)
print(x.shape)

torch.Size([8, 4, 256, 256])
torch.Size([8, 32, 254, 254])
torch.Size([8, 32, 127, 127])
torch.Size([8, 32, 125, 125])
torch.Size([8, 32, 62, 62])
torch.Size([8, 123008])
torch.Size([8, 120])


In [7]:
# Define U-Net architecture
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.middle = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.decoder = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, out_channels, kernel_size=2, stride=2)
        )

    def forward(self, x):
        x1 = self.encoder(x)
        x2 = self.middle(x1)
        x3 = self.decoder(x2)
        return x3

In [12]:
import torch

In [13]:
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=4, out_channels=13, init_features=32)

/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/mateuszbuda/brain-segmentation-pytorch/zipball/master" to /root/.cache/torch/hub/master.zip


In [ ]:
import torch.optim as optim

# Instantiate the model, loss function, and optimizer
# model = UNet(in_channels=4, out_channels=13)  # Adjust in_channels and out_channels based on your task
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()

    for data in image_tensors:
        inputs=data['image']
        targets=data['mask'][:,0,...]
        optimizer.zero_grad()
        outputs = model(inputs)
        print(inputs.shape,targets.shape,outputs.shape)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')


torch.Size([10, 4, 256, 256]) torch.Size([10, 256, 256]) torch.Size([10, 13, 256, 256])
torch.Size([10, 4, 256, 256]) torch.Size([10, 256, 256]) torch.Size([10, 13, 256, 256])
torch.Size([10, 4, 256, 256]) torch.Size([10, 256, 256]) torch.Size([10, 13, 256, 256])
torch.Size([10, 4, 256, 256]) torch.Size([10, 256, 256]) torch.Size([10, 13, 256, 256])
torch.Size([10, 4, 256, 256]) torch.Size([10, 256, 256]) torch.Size([10, 13, 256, 256])
torch.Size([10, 4, 256, 256]) torch.Size([10, 256, 256]) torch.Size([10, 13, 256, 256])
torch.Size([10, 4, 256, 256]) torch.Size([10, 256, 256]) torch.Size([10, 13, 256, 256])
torch.Size([10, 4, 256, 256]) torch.Size([10, 256, 256]) torch.Size([10, 13, 256, 256])
torch.Size([10, 4, 256, 256]) torch.Size([10, 256, 256]) torch.Size([10, 13, 256, 256])
torch.Size([10, 4, 256, 256]) torch.Size([10, 256, 256]) torch.Size([10, 13, 256, 256])
torch.Size([10, 4, 256, 256]) torch.Size([10, 256, 256]) torch.Size([10, 13, 256, 256])
torch.Size([10, 4, 256, 256]) to